In [ ]:
https://mlcourse.ai/book/topic10/topic10_gradient_boosting.html
http://nicolas-hug.com/pres_scipy_2019/#1
https://nicolas-hug.com/blog/gradient_boosting_descent#wait-what
https://inria.github.io/scikit-learn-mooc/python_scripts/parameter_tuning_manual.html

- Early stopping
- Cross-validation
- Hyperameter tuning
- Feature importance


Final function

Lab: apply this to a Kaggle competition